# <div align="center">Programa Nacional de Vigilância em Saúde em Navios de Cruzeiro</div>

<div align="center"><img src="https://i.imgur.com/FkLJ24t.png"></div>

A análise visa responder algumas perguntas sobre inspeção de navios de cruzeiro que faz parte do **Programa Nacional de Vigilância Sanitária para Navios de Cruzeiro** realizado pela **Anvisa** no perído de outubro até o mês de abril do ano subsquente. O objetivo deste programa é avaliar a segurança sanitária dos alimentos e da água para consumo nas embarcações além de acompanhar o Plano de Limpeza e Desinfecção nas instalações e áreas internas dos navios reforçando a prevenção de surtos de doenças no decorrer das viagens.

A Anvisa classifica os navios em níveis:

|Nível |Pontuação de Risco | Categoria |
|:---:|:---:|:---:|
| A | 0-150 | Excelente|
| B | 151-300 | Satisfatório|
| C | 301-450 | Aceitável|
| D | 450-... | Insatisfatório|

<br>

O schema *sql-dsa* foi criado e os dados foram inseridos no banco de dados através da **MySQL Benchmark**.

Nesta análise são mostradas as características de **selecão, ordenamento e filtragem** da linguagem **SQL**

**Dataset em formato .CSV está disponível:**
- [Anvisa](https://dados.anvisa.gov.br/dados/DADOS_ABERTOS_INSPECAO_NAVIO.csv)
- [Projeto](./DADOS_ABERTOS_INSPECAO_NAVIO.csv).

<br>

**Dicionário de dados em formato .PDF está disponível**
- [Anvisa](https://dados.anvisa.gov.br/dados/Documentacao_e_Dicionario_de_Dados_Navios_Cruzeiro.pdf)
- [Projeto](./Documentacao_e_Dicionario_de_Dados_Navios_Cruzeiro.pdf)

<br>

**Script Conexão Python-MySQL**
- [Jupyter Notebook]()

<br>

**Script para a criação da tabela tb.navios**
- [SQL](./cria_tabela_tb.navios.sql)

<hr>

## <a name="indice"></a> ÍNDICE

**Obs**: direcionamento do índice apenas quando este notebook é executado pelo Jupyter Notebook.

- [Conexão](#conexao)
- [Schema Anvisa e Tabela Navios](#tabela-anvisa)
- [Análise](#analise)
    1. [Total de Embarcações Diferentes](#1)
    2. [Número Total de Inspeções](#2)
    3. [Quais embarcações e quando ocorreu determinada classificação de risco para uma embarcação](#3)
    4. [Navios com avaliação de qualidade máxima](#4)
    5. [As 10 piores ocorrências de inspeção](#5)
    6. [Meios: Navios com padrões B e C e pontuação de risco no range de 151 a 450](#6)
    7. [Extremos: Navios com padrões A e D e pontuação de risco FORA do range de 151 a 450](#7)
    8. [COVID-19: Embarcações com iniciais A e B, índice de conformidade B ou C e conformidade <= 90% antes da pandemia](#8)

<hr>

## <a name="conexao"></a> Conexão

In [ ]:
#Conexão Jupyter Notebook-MySQL

import sqlalchemy

user = input('Digite o usuário: ')
psw = input('Digite a senha: ')
host = input('Digite o host: ')
porta = input('Digite a porta de conexão: ')
conexao = 'mysql+mysqlconnector://'+user+':'+psw+'@'+host+':'+porta

%reload_ext sql
%sql $conexao


<hr>

## <a name="tabela-anvisa"></a>Schema Anvisa e Tabela Navios

In [ ]:
%%sql

CREATE SCHEMA IF NOT EXISTS sql01_anvisa
CHARACTER SET = utf8mb4;

CREATE TABLE IF NOT EXISTS `sql01_anvisa`.`tb_navios`(
  `nome_navio` VARCHAR(50) NULL,
  `mes_ano` VARCHAR(10) NULL,
  `classificacao_risco` VARCHAR(15) NULL,
  `indice_conformidade` VARCHAR(15) NULL,
  `pontuacao_risco` INT NULL,
  `temporada` VARCHAR(200) NULL);

  USE sql01_anvisa;

<hr>

## <a name="analise"></a> Análise

### 1

>**Inicialmente podemos contabilizar quantas embarcações diferentes foram inspecionadas ao longo de todo registro de inspeções**

[Voltar ao início](#indice)

In [ ]:
%%sql 

SELECT 
    COUNT(DISTINCT nome_navio) AS 'Quantidade de Navios Inspecionados'
FROM
    tb_navios;

### 2

> **Contabilizar o número total de inspeções ao longo do período**

[Voltar ao início](#indice)

In [ ]:
%%sql

SELECT 
    COUNT(*) AS 'Quantidade Total de Inspeções'
FROM
    tb_navios;

### 3

> **Neste dataset, várias embarcações foram inspecionadas mais de uma vez. Podemos saber quais navios, pelo menos uma vez, receberam uma determinada classificação a risco e quando isso ocorreu.**

[Voltar ao início](#indice)

In [ ]:
%%sql

SELECT DISTINCT
    nome_navio AS 'Nome do Navio',
    classificacao_risco AS 'Classificação de Risco',
    mes_ano AS 'Data da Inspeção'
FROM
    tb_navios
WHERE
    classificacao_risco = 'A'
ORDER BY nome_navio ASC , mes_ano DESC;

### 4

> **Outra indagação é sobre quais ocasiões as embarcações conseguiram alcançar os requisitos plenos da Anvisa, ou seja, índice de conformidade de 100% e pontuação de risco 0 pelo menos uma vez? Como os resultados de índice e pontuação serão idênticos, podemos ordenar pelo nome do navio na ordem A-Z (ou crescente) e a data de ocorrência da inspeção.**

[Voltar ao início](#indice)

In [ ]:
%%sql

SELECT 
    nome_navio AS 'Nome do Navio',
    indice_conformidade AS 'Índice de Conformidade',
    pontuacao_risco AS 'Pontuação de Risco',
    mes_ano AS 'Data da Inspeção'
FROM
    tb_navios
WHERE
    indice_conformidade = 100
        AND pontuacao_risco = 0
ORDER BY mes_ano , nome_navio;

### 5

> **E quais seriam as embarcações com os piores indicadores? Selecionamos os 10 piores resultados da classificaçao D juntamente com a data da ocorrência.**

[Voltar ao início](#indice)

In [ ]:
%%sql

SELECT DISTINCT
    nome_navio AS 'Nome do Navio',
    pontuacao_risco AS 'Pontuação de Risco',
    mes_ano AS 'Data da Inspeção'
FROM
    tb_navios
WHERE
    pontuacao_risco > 450
ORDER BY pontuacao_risco DESC
LIMIT 10;


### 6

> **Ao retornar valores em um intervalo determinado podemos selecionar as embarcações que receberam padrões B e C pelo menos uma vez retornando um range de 151 até 450 pontos de risco.**

[Voltar ao início](#indice)

In [ ]:
%%sql

SELECT 
    nome_navio AS 'Nome do Navio',
    pontuacao_risco AS 'Pontuação de Risco',
    mes_ano AS 'Data da Inspeção'
FROM
    tb_navios
WHERE
    pontuacao_risco BETWEEN 151 AND 450
ORDER BY nome_navio ASC , mes_ano DESC;

### 7

> **O inverso também se aplica caso não necessitemos que determinado grupo seja selecionado. Agora vamos retornar apenas os navios que já estiveram no padrão A ou D**

[Voltar ao início](#indice)

In [ ]:
%%sql

SELECT 
    nome_navio AS 'Nome do Navio',
    pontuacao_risco AS 'Pontuação de Risco',
    mes_ano AS 'Data da Inspeção'
FROM
    tb_navios
WHERE
    pontuacao_risco NOT BETWEEN 151 AND 450
ORDER BY pontuacao_risco , nome_navio , mes_ano DESC;

### 8

> **Quais embarcações que comecem com a letra 'A' ou 'B' já fizeram parte, pelo menos uma vez, dos níveis B ou C e tiveram índices de conformidade menor ou igual a 95% em inspeções antes do ano em que a pandemia do COVID-19 chegou ao Brasil (2020)?**

[Voltar ao início](#indice)

In [ ]:
%%sql

SELECT 
    nome_navio AS 'Nome do Navio',
    mes_ano AS 'Data da Inspeção',
    classificacao_risco AS 'Classificação de Risco',
    indice_conformidade AS 'Ìndice de Conformidade',
    temporada AS Temporada
FROM
    tb_navios
WHERE
    (classificacao_risco = 'B'
        OR classificacao_risco = 'C')
        AND indice_conformidade <= 95
        AND mes_ano < '01/2020'
        AND (nome_navio LIKE 'A%'
        OR nome_navio LIKE 'B%')
ORDER BY mes_ano , indice_conformidade , nome_navio;